Задание к лекции "Основы веб-скрапинга и работы с API"

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [34]:
import pandas as pd
import requests
import time
import regex
from bs4 import BeautifulSoup
url = 'https://habr.com/ru/all/'
keywords = ['Python', 'python', 'Парсинг','парсинг']


In [35]:
def get_all_info(url, keywords, pages):
    habr_posts = pd.DataFrame()
    
    for i in range(pages):
        params = 'page' + str(i + 1)
        res = requests.get(url+params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        title_blocks = soup.find_all('h2', class_='post__title')
        text_blocks = soup.find_all('div', class_='post__text post__text-html post__text_v1')
        time_block = soup.find_all('span', class_='post__time')
        all_titles = list(map(lambda x: x.find('a', class_='post__title_link').text, title_blocks))
        all_linkes = list(map(lambda x: x.find('a').get('href'), title_blocks))
        all_text = list(map(lambda x: x.get_text(),text_blocks))
        all_time = list(map(lambda x: x.get_text(), time_block))
        all_info = [list(tup) for tup in zip(all_time, all_titles, all_text, all_linkes)]
        for j in range(len(all_info)):
            if regex.search(r"\L<words>", all_info[j][1], words=keywords) or\
            regex.search(r"\L<words>", all_info[j][2], words=keywords):
                row = {'time': all_info[j][0], 'title': all_info[j][1], 'text': all_info[j][2], 'link': all_info[j][3]}
                habr_posts = pd.concat([habr_posts, pd.DataFrame([row])]) 
                 
    return habr_posts

all_links = get_all_info(url, keywords, 30)
all_links

,time,title,text,link
0,сегодня в 16:05,Attack-defence для бизнеса: разбираем задания ...,\n Доброго времени суток читающий. Мне хо...,https://habr.com/ru/company/bizone/blog/525406/
0,сегодня в 16:04,Вскрытие покажет: Решаем лёгкий crackme и пише...,\nВ преддверии старта курса «Python Developer....,https://habr.com/ru/post/526136/
0,сегодня в 15:54,Побег из песочницы с Python,\nНа прошлой неделе команда Google Project Zer...,https://habr.com/ru/company/otus/blog/526132/
0,вчера в 22:33,"Шаблонные функции в Python, которые могут выпо...",\n\n\r\nСейчас практически каждый разработчик ...,https://habr.com/ru/post/525998/
0,29 октября 2020 в 18:27,"«Здравствуй, новый хозяин». Как мы учили умные...",\n В данной статье рассматривается ситуац...,https://habr.com/ru/company/madrobots/blog/525...
0,29 октября 2020 в 14:26,Первый открытый xOwner Club в Райффайзенбанке ...,\n На конференции North Bay Python в 2018...,https://habr.com/ru/company/raiffeisenbank/blo...
0,28 октября 2020 в 12:05,Как быстро интегрируемые клинические данные по...,\n Давно хотел попробовать свои силы в ко...,https://habr.com/ru/company/cloud4y/blog/525388/
0,28 октября 2020 в 11:45,"Бьярне Страуструп о контроле над языком, удале...","\n\n\r\nУ всех нас только и разговоров, что о ...",https://habr.com/ru/company/jugru/blog/525288/
0,27 октября 2020 в 19:21,"Ральф Баер: пиксель, пиксель, огуречик",\n\r\nРеализуем и сравниваем 4 популярных опти...,https://habr.com/ru/company/it-grad/blog/525300/
0,26 октября 2020 в 21:35,"Рудиментарные дела. Кого, чему, зачем и что не...","\n Привет, друзья! Хорошая новость для мо...",https://habr.com/ru/post/525168/
